# 梯度优化

## 基本概念

### 权重

**权重**： 又称为可训练参数（trainable parameter），分别对应 kernel 和 bias 属性。
**随机初始化（random initialization）**： 赋值为权重矩阵取较小的随机值  
**训练/学习**： 逐渐调节权重的过程  

具体步骤如下：

- 抽取训练样本 x 和对应目标 y 组成的数据批量。
- 在 x 上运行网络［这一步叫作前向传播（forward pass）］，得到预测值 y_pred。
- 计算网络在这批数据上的损失，用于衡量 y_pred 和 y 之间的距离。
- 更新网络的所有权重，使网络在这批数据上的损失略微下降。

如何更新权重：
一种简单的解决方案是，保持网络中其他权重不变，只考虑某个标量系数，让其尝试不同的取值。
一种更好的方法是利用网络中所有运算都是可微（differentiable）的这一事实，计算损失相对于网络系数的梯度（gradient），然后向梯度的反方向改变系数，从而使损失降低。

### 导数概念

假设有一个连续的光滑函数 f(x) = y，将实数 x 映射为另一个实数 y。由于函数是连续的，x 的微小变化只能导致 y 的微小变化。假设 x 增大了一个很小的因子 epsilon_x，这导致 y 也发生了很小的变化，即 epsilon_y：

$f(x+epsilon\_x) = y + epsilon\_y$

如果变化再某个点p附近,因子 epsilon_x足够小，将 f 近似为斜率为 a 的线性函数
$f(x+epsilon\_x) = y + a * epsilon\_x$

斜率 a 被称为 f 在 p 点的**导数（derivative）**

![导数的概念](imgs/05.jpg)

导数完全描述了改变 x 后 f(x) 会如何变化。如果你希望减小 f(x) 的值，只需将 x 沿着导数的反方向移动一小步。


### 梯度概念

梯度（gradient）是张量运算的导数。它是导数这一概念向多元函数导数的推广。多元函数是以张量作为输入的函数。
如果输入数据 x 和 y 保持不变，那么这可以看作将 W 映射到损失值的函数。
$loss_value = f(W)$
假设 W 的当前值为 W0。f 在 W0 点的导数是一个张量 gradient(f)(W0)，其形状与 W 相同，每个系数 gradient(f)(W0)\[i,j] 表示改变 W0[i,j] 时 loss_value 变化的方向和大小。张量 gradient(f)(W0) 是函数 f(W) = loss_value 在 W0 的导数。

单变量函数 f(x) 的导数可以看作函数 f 曲线的斜率。同样，gradient(f)(W0) 也可以看作表示 f(W) 在 W0 附近**曲率（curvature）**的张量。 

对于张量的函数 f(W)，你也可以通过将 W 向梯度的反方向移动来减小 f(W)，比如 W1  = W0 - step  *  gradient(f)(W0)，其中 step 是一个很小的比例因子。 

### 随机梯度下降

给定一个可微函数，理论上可以用解析法找到它的最小值：函数的最小值是导数为 0 的点，因此你只需找到所有导数为 0 的点，然后计算函数在其中哪个点具有最小值。

将这一方法应用于神经网络，就是用解析法求出最小损失函数对应的所有权重值。可以通过对方程 gradient(f)(W)  =  0 求解 W 来实现这一方法，，但对于实际的神经网络是无法求解的，因为参数的个数不会少于几千个，而且经常有上千万个。
具体的步骤：
(1)  抽取训练样本 x 和对应目标 y 组成的数据批量。
(2)  在 x 上运行网络，得到预测值 y_pred。
(3)  计算网络在这批数据上的损失，用于衡量 y_pred 和 y 之间的距离。
(4)  计算损失相对于网络参数的梯度［一次反向传播（backward pass）］。
(5)  将参数沿着梯度的反方向移动一点，比如 W -= step * gradient，从而使这批数据上的损失减小一点。


### 几种梯度下降法的比较

先来快速看一下 BGD，SGD，MBGD 的定义，
- 当每次是对整个训练集进行梯度下降的时候，就是 batch 梯度下降，
- 当每次只对一个样本进行梯度下降的时候，是 stochastic 梯度下降，
- 当每次处理样本的个数在上面二者之间，就是 mini batch 梯度下降。

**mini-batch 梯度下降法**：
t 代表第几个子集，从 1 到 5000，因为划分后，一共有 5000 个子集，
1. 对每个子集，先进行前向计算，从第一层网络到最后一层输出层因为 batch 梯度下降是对整个数据集进行处理，所以不需要角标，而 mini batch 这里需要对 x 加上角标，代表的是第几个子集。
2. 接下来计算当前子集的损失函数，因为子集中一共有 1000 个样本，所以这里要除以 1000。损失函数也是有上角标，和第几个子集相对应。
3. 然后进行反向传播，计算损失函数 J 的梯度。
4. 最后更新参数。


**mini-batch 梯度下降法优点**

- 和 batch 梯度下降相比，使用 mini batch 梯度下降更新参数更快，有利于更鲁棒地收敛，避免局部最优。
- 和 stochastic 梯度下降相比，使用 mini batch 梯度下降的计算效率更高，可以帮助快速训练模型。


**mini-batch 梯度下降具体过程**

在 mini batch 梯度下降中，并不是每一批的成本都是下降的，
因为每次迭代都是在训练不同的子集，所以展示在图像上就是，整体走势是下降的，但是会有更多的噪音。
噪音的原因是，如果是比较容易计算的子集，需要的成本就会低一些，遇到难算的子集，成本就要高一些。
![噪音比较](imgs/06.jpg)

- 蓝色：为 batch 梯度下降，即 mini batch size ＝ m，
- 紫色：为 stochastic 梯度下降，即 mini batch size ＝ 1，
- 绿色：为 mini batch 梯度下降，即 1 < mini batch size < m


![噪音比较](imgs/07.jpg)

Batch gradient descent，噪音少一些，幅度大一些。
BGD 的缺点是，每次对整个训练集进行处理，那么数量级很大的时候耗费时间就会比较长。

Stochastic gradient descent ，因为每次只对一个样本进行梯度下降，所以大部分时候是向着最小值靠近的，但也有一些是离最小值越来越远，因为那些样本恰好指向相反的方向。所以看起来会有很多噪音，但整体趋势是向最小值逼近。

但 SGD 永远不会收敛，它只会在最小值附近不断的波动，不会到达也不会在此停留。SGD 的噪音，可以通过调节学习率来改善，但是它有一个很大的缺点，就是不能通过进行向量化来进行加速，因为每次都只是对一个样本进行处理。

Mini Batch gradient descent 的每个子集的大小正好位于两种极端情况的中间。
那就有两个好处，一个是可以进行向量化。另一个是不用等待整个训练集训练完就可以进行后续的工作。
MBGD 的成本函数的变化，不会一直朝着最小值的方向前进，但和 SGD 相比，会更持续地靠近最小值。



